Import library

ライブラリーをインポートする

In [1]:
import pandas as pd
import time
import csv
from selenium import webdriver
from bs4 import BeautifulSoup as soup
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

This is the URL used to search for the toll fees. 

このURLで高速料金を探す

In [2]:
options = Options()
options.add_argument('--headless')
url = "http://search.w-nexco.co.jp/route.php"

This will read the csv file of the list and delete the empty rows with no IC name available. It will also create a new list (full_**) to input all the toll fees inside.

CSVファイルを読み込み、空セルを削除。リスト(full_**)を作る

In [3]:
#read CSV
df = pd.read_csv('201807_ryokin1.csv')
df_edit = df.dropna(subset=['入口','出口'])
full_kei = []
full_normal = []
full_chugata = []
full_ogata = []
full_tokugata = []

Run the whole program and iterate through each row. A time.sleep is put to add time for response. Prevent from being recognized as bot

プログラムを動かす。車種区分に車種によって変更してください。

This program is for the kei car.　これは**軽・自動二輪**の車種区分

In [4]:
for index, row in df_edit.iterrows():
    
    #open the browser and navigate to the page
    #ブラウザ開いてURLに接続
    #driver = webdriver.Chrome(chrome_options=options)
    driver = webdriver.Chrome()
    driver.get(url)

    #fill in the form
    
    #日時
    #set date (2018/7/10)
    driver.find_element_by_id("calImgDiv").click()
    driver.find_element_by_id("day_2018_7_10_2_0").click()
    #set time (午前10時)
    select_hour = Select(driver.find_element_by_id("sl_hour_id")).select_by_value("10")
    time.sleep(1)
    select_min = Select(driver.find_element_by_id("sl_min_id")).select_by_value("0")
    time.sleep(1)
    
    #出発IC
    in_field = driver.find_element_by_name("fnm")
    in_field.send_keys(row['入口'])
    time.sleep(1)
    #到着IC
    out_field = driver.find_element_by_name("tnm")
    out_field.send_keys(row['出口'])
    time.sleep(1)
    
    #車種区分
    #Type of car: ("1"-軽・自動二輪, "2"-普通車, "3"-中型車, "4"-大型車, "5"- 特大車) 
    select_car_type = Select(driver.find_element_by_name("cartyp")).select_by_value("1")
    time.sleep(1)
    
    
    
    #通らない道路　（二つしかセットできない）
    #set detour (only 2 can be set)
    #"G1110"- C3 外環道, "G6000"-首都高速
    select_detour1 = Select(driver.find_element_by_id("detour1_id")).select_by_value("G1110")
    time.sleep(1)
    select_detour2 = Select(driver.find_element_by_id("detour2_id")).select_by_value("G6000")
    
    #検索する
    driver.find_element_by_css_selector(".submit-btn").send_keys("\n")
    time.sleep(3)
    driver.find_element_by_css_selector(".submit-btn").send_keys("\n")
    time.sleep(2)
        
    #wait for page to load and click ETC料金順
    #ページを待つとETC料金順をクリック
    time.sleep(3)
    driver.find_element_by_id("pritab9").click()
    
    #wait for page to load and get the toll fee
    #ページを待つとデータを取る
    time.sleep(3)
    
    #通常(現金)
    normal_toll = driver.find_element_by_css_selector("span.toll-normal").get_attribute("innerText")
    #ETC
    etc_toll = driver.find_element_by_css_selector("span.toll-etc").get_attribute("innerText")
    #ETC2.0
    etc2_toll = driver.find_element_by_css_selector("span.toll-etc2").get_attribute("innerText")
    #休日
    kyujitsu_toll = driver.find_element_by_xpath('//*[@class="etc-info pc-stay pos-etc_box"]/div[2]/div/div/dl[1]/dd/span').get_attribute("innerText")
    #深夜
    shinya_toll = driver.find_element_by_xpath('//*[@class="etc-info pc-stay pos-etc_box"]/div[2]/div/div/dl[2]/dd/span').get_attribute("innerText")
    
    time.sleep(5)

    #close browser and session
    #ブラウザを閉じる
    driver.quit()
    
    #collected toll fess is put inside the list and made into list of list
    #取ったデータをリストに入れる
    toll = [row['入口'],row['出口'],normal_toll, etc_toll, etc2_toll, shinya_toll, kyujitsu_toll]
    full_kei.append(toll)
    
    #wait time before next session
    #次のセッションをスタートする待つ時間
    time.sleep(10)

This program is for the normal car.　これは**普通車**の車種区分

In [5]:
for index, row in df_edit.iterrows():
    
    #open the browser and navigate to the page
    #ブラウザ開いてURLに接続
    #driver = webdriver.Chrome(chrome_options=options)
    driver = webdriver.Chrome()
    driver.get(url)

    #fill in the form
    
    #日時
    #set date (2018/7/10)
    driver.find_element_by_id("calImgDiv").click()
    driver.find_element_by_id("day_2018_7_10_2_0").click()
    #set time (午前10時)
    select_hour = Select(driver.find_element_by_id("sl_hour_id")).select_by_value("10")
    time.sleep(1)
    select_min = Select(driver.find_element_by_id("sl_min_id")).select_by_value("0")
    time.sleep(1)
    
    #出発IC
    in_field = driver.find_element_by_name("fnm")
    in_field.send_keys(row['入口'])
    time.sleep(1)
    #到着IC
    out_field = driver.find_element_by_name("tnm")
    out_field.send_keys(row['出口'])
    time.sleep(1)
    
    #車種区分
    #Type of car: ("1"-軽・自動二輪, "2"-普通車, "3"-中型車, "4"-大型車, "5"- 特大車) 
    select_car_type = Select(driver.find_element_by_name("cartyp")).select_by_value("2")
    time.sleep(1)
    
    #通らない道路　（二つしかセットできない）
    #set detour (only 2 can be set)
    #"G1110"- C3 外環道, "G6000"-首都高速
    select_detour1 = Select(driver.find_element_by_id("detour1_id")).select_by_value("G1110")
    time.sleep(1)
    select_detour2 = Select(driver.find_element_by_id("detour2_id")).select_by_value("G6000")
    
    #検索する
    driver.find_element_by_css_selector(".submit-btn").send_keys("\n")
    time.sleep(3)
    driver.find_element_by_css_selector(".submit-btn").send_keys("\n")
    time.sleep(2)
        
    #wait for page to load and click ETC料金順
    #ページを待つとETC料金順をクリック
    time.sleep(3)
    driver.find_element_by_id("pritab9").click()
    
    #wait for page to load and get the toll fee
    #ページを待つとデータを取る
    time.sleep(3)
    
    #通常(現金)
    normal_toll = driver.find_element_by_css_selector("span.toll-normal").get_attribute("innerText")
    #ETC
    etc_toll = driver.find_element_by_css_selector("span.toll-etc").get_attribute("innerText")
    #ETC2.0
    etc2_toll = driver.find_element_by_css_selector("span.toll-etc2").get_attribute("innerText")
    #休日
    kyujitsu_toll = driver.find_element_by_xpath('//*[@class="etc-info pc-stay pos-etc_box"]/div[2]/div/div/dl[1]/dd/span').get_attribute("innerText")
    #深夜
    shinya_toll = driver.find_element_by_xpath('//*[@class="etc-info pc-stay pos-etc_box"]/div[2]/div/div/dl[2]/dd/span').get_attribute("innerText")
    
    time.sleep(5)

    #close browser and session
    #ブラウザを閉じる
    driver.quit()
    
    #collected toll fess is put inside the list and made into list of list
    #取ったデータをリストに入れる
    toll = [row['入口'],row['出口'],normal_toll, etc_toll, etc2_toll, shinya_toll, kyujitsu_toll]
    full_normal.append(toll)
    
    #wait time before next session
    #次のセッションをスタートする待つ時間
    time.sleep(10)

This program is for the chugata.　これは**中型車**の車種区分

In [6]:
for index, row in df_edit.iterrows():
    
    #open the browser and navigate to the page
    #ブラウザ開いてURLに接続
    driver = webdriver.Chrome(chrome_options=options)
    driver.get(url)

    #fill in the form
    #検索フォーム入力
    
    #日時
    #set date (2018/7/10)
    driver.find_element_by_id("calImgDiv").click()
    driver.find_element_by_id("day_2018_7_10_2_0").click()
    #set time (午前10時)
    select_hour = Select(driver.find_element_by_id("sl_hour_id")).select_by_value("10")
    time.sleep(1)
    select_min = Select(driver.find_element_by_id("sl_min_id")).select_by_value("0")
    time.sleep(1)
    
    #出発IC
    in_field = driver.find_element_by_name("fnm")
    in_field.send_keys(row['入口'])
    time.sleep(1)
    #到着IC
    out_field = driver.find_element_by_name("tnm")
    out_field.send_keys(row['出口'])
    time.sleep(1)
    
    #車種区分
    #Type of car: ("1"-軽・自動二輪, "2"-普通車, "3"-中型車, "4"-大型車, "5"- 特大車) 
    select_car_type = Select(driver.find_element_by_name("cartyp")).select_by_value("3")
    time.sleep(1)
    
    #通らない道路　（二つしかセットできない）
    #set detour (only 2 can be set)
    #"G1110"- C3 外環道, "G6000"-首都高速
    select_detour1 = Select(driver.find_element_by_id("detour1_id")).select_by_value("G1110")
    time.sleep(1)
    select_detour2 = Select(driver.find_element_by_id("detour2_id")).select_by_value("G6000")
    
    #検索する
    driver.find_element_by_css_selector(".submit-btn").send_keys("\n")
    time.sleep(3)
    driver.find_element_by_css_selector(".submit-btn").send_keys("\n")
    time.sleep(2)
        
    #wait for page to load and click ETC料金順
    #ページを待つとETC料金順をクリック
    time.sleep(3)
    driver.find_element_by_id("pritab9").click()
    
    #wait for page to load and get the toll fee
    #ページを待つとデータを取る
    time.sleep(3)
    
    #通常(現金)
    normal_toll = driver.find_element_by_css_selector("span.toll-normal").get_attribute("innerText")
    #ETC
    etc_toll = driver.find_element_by_css_selector("span.toll-etc").get_attribute("innerText")
    #ETC2.0
    etc2_toll = driver.find_element_by_css_selector("span.toll-etc2").get_attribute("innerText")
    #深夜
    shinya_toll = driver.find_element_by_xpath('//*[@class="etc-info pc-stay pos-etc_box"]/div[2]/div/div/dl[1]/dd/span').get_attribute("innerText")
    
    time.sleep(5)

    #close browser and session
    #ブラウザを閉じる
    driver.quit()
    
    #collected toll fess is put inside the list and made into list of list
    #取ったデータをリストに入れる
    toll = [row['入口'],row['出口'],normal_toll, etc_toll, etc2_toll, shinya_toll]
    full_chugata.append(toll)
    
    #wait time before next session
    #次のセッションをスタートする待つ時間
    time.sleep(10)

This program is for the ogata.　これは**大型車**の車種区分

In [7]:
for index, row in df_edit.iterrows():
    
    #open the browser and navigate to the page
    #ブラウザ開いてURLに接続
    driver = webdriver.Chrome(chrome_options=options)
    driver.get(url)

    #fill in the form
    #検索フォーム入力
    
    #日時
    #set date (2018/7/10)
    driver.find_element_by_id("calImgDiv").click()
    driver.find_element_by_id("day_2018_7_10_2_0").click()
    #set time (午前10時)
    select_hour = Select(driver.find_element_by_id("sl_hour_id")).select_by_value("10")
    time.sleep(1)
    select_min = Select(driver.find_element_by_id("sl_min_id")).select_by_value("0")
    time.sleep(1)
    
    #出発IC
    in_field = driver.find_element_by_name("fnm")
    in_field.send_keys(row['入口'])
    time.sleep(1)
    #到着IC
    out_field = driver.find_element_by_name("tnm")
    out_field.send_keys(row['出口'])
    time.sleep(1)
    
    #車種区分
    #Type of car: ("1"-軽・自動二輪, "2"-普通車, "3"-中型車, "4"-大型車, "5"- 特大車) 
    select_car_type = Select(driver.find_element_by_name("cartyp")).select_by_value("4")
    time.sleep(1)
    
    #通らない道路　（二つしかセットできない）
    #set detour (only 2 can be set)
    #"G1110"- C3 外環道, "G6000"-首都高速
    select_detour1 = Select(driver.find_element_by_id("detour1_id")).select_by_value("G1110")
    time.sleep(1)
    select_detour2 = Select(driver.find_element_by_id("detour2_id")).select_by_value("G6000")
    
    #検索する
    driver.find_element_by_css_selector(".submit-btn").send_keys("\n")
    time.sleep(3)
    driver.find_element_by_css_selector(".submit-btn").send_keys("\n")
    time.sleep(2)
        
    #wait for page to load and click ETC料金順
    #ページを待つとETC料金順をクリック
    time.sleep(3)
    driver.find_element_by_id("pritab9").click()
    
    #wait for page to load and get the toll fee
    #ページを待つとデータを取る
    time.sleep(3)
    
    #通常(現金)
    normal_toll = driver.find_element_by_css_selector("span.toll-normal").get_attribute("innerText")
    #ETC
    etc_toll = driver.find_element_by_css_selector("span.toll-etc").get_attribute("innerText")
    #ETC2.0
    etc2_toll = driver.find_element_by_css_selector("span.toll-etc2").get_attribute("innerText")
    #深夜
    shinya_toll = driver.find_element_by_xpath('//*[@class="etc-info pc-stay pos-etc_box"]/div[2]/div/div/dl[1]/dd/span').get_attribute("innerText")
    
    time.sleep(5)

    #close browser and session
    #ブラウザを閉じる
    driver.quit()
    
    #collected toll fess is put inside the list and made into list of list
    #取ったデータをリストに入れる
    toll = [row['入口'],row['出口'],normal_toll, etc_toll, etc2_toll, shinya_toll]
    full_ogata.append(toll)
    
    #wait time before next session
    #次のセッションをスタートする待つ時間
    time.sleep(10)

This program is for the tokugata.　これは**特大車**の車種区分

In [8]:
for index, row in df_edit.iterrows():
    
    #open the browser and navigate to the page
    #ブラウザ開いてURLに接続
    driver = webdriver.Chrome(chrome_options=options)
    driver.get(url)

    #fill in the form
    #検索フォーム入力
    
    #日時
    #set date (2018/7/10)
    driver.find_element_by_id("calImgDiv").click()
    driver.find_element_by_id("day_2018_7_10_2_0").click()
    #set time (午前10時)
    select_hour = Select(driver.find_element_by_id("sl_hour_id")).select_by_value("10")
    time.sleep(1)
    select_min = Select(driver.find_element_by_id("sl_min_id")).select_by_value("0")
    time.sleep(1)
    
    #出発IC
    in_field = driver.find_element_by_name("fnm")
    in_field.send_keys(row['入口'])
    time.sleep(1)
    #到着IC
    out_field = driver.find_element_by_name("tnm")
    out_field.send_keys(row['出口'])
    time.sleep(1)
    
    #車種区分
    #Type of car: ("1"-軽・自動二輪, "2"-普通車, "3"-中型車, "4"-大型車, "5"- 特大車) 
    select_car_type = Select(driver.find_element_by_name("cartyp")).select_by_value("5")
    time.sleep(1)
    
    #通らない道路　（二つしかセットできない）
    #set detour (only 2 can be set)
    #"G1110"- C3 外環道, "G6000"-首都高速
    select_detour1 = Select(driver.find_element_by_id("detour1_id")).select_by_value("G1110")
    time.sleep(1)
    select_detour2 = Select(driver.find_element_by_id("detour2_id")).select_by_value("G6000")
    
    #検索する
    driver.find_element_by_css_selector(".submit-btn").send_keys("\n")
    time.sleep(3)
    driver.find_element_by_css_selector(".submit-btn").send_keys("\n")
    time.sleep(2)
        
    #wait for page to load and click ETC料金順
    #ページを待つとETC料金順をクリック
    time.sleep(3)
    driver.find_element_by_id("pritab9").click()
    
    #wait for page to load and get the toll fee
    #ページを待つとデータを取る
    time.sleep(3)
    
    #通常(現金)
    normal_toll = driver.find_element_by_css_selector("span.toll-normal").get_attribute("innerText")
    #ETC
    etc_toll = driver.find_element_by_css_selector("span.toll-etc").get_attribute("innerText")
    #ETC2.0
    etc2_toll = driver.find_element_by_css_selector("span.toll-etc2").get_attribute("innerText")
    #深夜
    shinya_toll = driver.find_element_by_xpath('//*[@class="etc-info pc-stay pos-etc_box"]/div[2]/div/div/dl[1]/dd/span').get_attribute("innerText")
    
    time.sleep(5)

    #close browser and session
    #ブラウザを閉じる
    driver.quit()
    
    #collected toll fess is put inside the list and made into list of list
    #取ったデータをリストに入れる
    toll = [row['入口'],row['出口'],normal_toll, etc_toll, etc2_toll, shinya_toll]
    full_tokugata.append(toll)
    
    #wait time before next session
    #次のセッションをスタートする待つ時間
    time.sleep(10)

Change list into Pandas DataFrame. Make another dataframe to exclude all unrelated symbols (円,分, etc.)

リストをPandasデータフレームに変更。もう一つのデータフレームを作り要らない記号を削除。

In [20]:
#header for dataframe
#データのヘッダー
title_1 = ['入口', '出口', '通常（現金）', 'ETC', 'ETC2.0', '深夜', '休日']
title_2 = ['入口', '出口', '通常（現金）', 'ETC', 'ETC2.0', '深夜']

pd_toll_kei = pd.DataFrame(full_kei, columns = title_1)
all_kei = pd_toll_kei[pd_toll_kei.columns[2:]].replace('[\$,円,分]', '', regex=True).astype(int)

pd_toll_normal = pd.DataFrame(full_normal, columns = title_1)
all_normal = pd_toll_normal[pd_toll_normal.columns[2:]].replace('[\$,円,分]', '', regex=True).astype(int)

pd_toll_chugata = pd.DataFrame(full_chugata, columns = title_2)
all_chugata = pd_toll_chugata[pd_toll_chugata.columns[2:]].replace('[\$,円,分]', '', regex=True).astype(int)

pd_toll_ogata = pd.DataFrame(full_ogata, columns = title_2)
all_ogata = pd_toll_ogata[pd_toll_ogata.columns[2:]].replace('[\$,円,分]', '', regex=True).astype(int)

pd_toll_tokugata = pd.DataFrame(full_tokugata, columns = title_2)
all_tokugata = pd_toll_tokugata[pd_toll_tokugata.columns[2:]].replace('[\$,円,分]', '', regex=True).astype(int)

Export into Microsoft Excel file.

エクセルにエクスポートする

In [21]:
with pd.ExcelWriter('full toll fee.xlsx') as writer:
    pd_toll_kei.to_excel(writer, sheet_name='pd toll(kei)')
    all_kei.to_excel(writer, sheet_name='all kei')
    pd_toll_normal.to_excel(writer, sheet_name='pd toll(normal)')
    all_normal.to_excel(writer, sheet_name='all normal')
    pd_toll_chugata.to_excel(writer, sheet_name='pd toll(chugata)')
    all_chugata.to_excel(writer, sheet_name='all chugata')
    pd_toll_ogata.to_excel(writer, sheet_name='pd toll(ogata)')
    all_ogata.to_excel(writer, sheet_name='all ogata')
    pd_toll_tokugata.to_excel(writer, sheet_name='pd toll(tokugata)')
    all_tokugata.to_excel(writer, sheet_name='all tokugata')